![Title](Images/cisco.png)

# Lab - Image Processing Change Detection

### Objectives
<p>
<li>**Part 1: Learn About Computer Imaging**</li>
<li>**Part 2: Capture Images**</li>
<li>**Part 3: Change Detection**</li>
<li>**Part 4: Explanation and Application**</li>

### Scenario/Background
Detecting moving objects in a scene is one of the most important tasks in computer vision. It is useful for video surveillance, for instance, but it is usually the basis for other, more complex tasks.

Usually the way this task is performed is by building a model of an empty scene and comparing each newly captured frame to it. When the model of the empty scene is different from the current scene, it means that something in the image has changed. 

In this lab, we are going to use a video camera attached to the Raspberry Pi to capture videos and images. We will build a very simple background model that will allow us to detect changes in the scene that the camera is capturing. We are capturing the images and converting them into matrices of numbers that can be operated on mathematically. This is different from capturing images and saving them as files.
### Required Resources
<p>
* 1 PC with Internet access
* Raspberry Pi version 2 or higher
* Python libraries: numpy, cv2, picamera, time, pyplot, numpy
* Picamera V2 module

The Raspberry Pi camera module was connected and installed in a previous lab. This lab assumes that the installation and testing of the camera module have previously been completed.

# Your name here:
# Your student ID here:

## Part 1: Learn About Computer Imaging

Let's have a look at the kind of data we are going to work with: digital images and videos.

A digital image is a matrix composed of pixels. Each pixel corresponds to a position inside of the image and contains information regarding the color of the image at that location. If the image is grayscale, each pixel contains a number that identifies the `level of gray` in that position. If we are looking at a color image, then each pixel contains a vector that encodes a color according to a certain color map. For instance, if we are using the RGB color map, each color is encoded by a group of three numbers. The numbers reprent the quantity of red, blue and green that makes up the color of the pixel. 

So, if we want to compare what we see versus what the computer sees, it would be something like this:

![title](Images/lisa_n.png)

Each pixel in an image that is using the RGB color space has three RGB values that range from 0 to 255 that define the color of that pixel. An image that is 256 pixels high and 256 pixles wide has 65,536 pixels. Each pixel has 24 bits of color information associated with it in an RGB color space. (The 24 bits consist of eight bits each for the red, green, and blue color channels.) The figure shows a numeric array that represents a 16x16 pixel area of the larger image. The values in the numeric array for each pixel represent the red value in that pixel. The pixels also have green and blue values that are not shown for clarity. The combination of the red, green, and blue values can depict 16,777,216 color variations (256 to the third power).

Computer vision is a demanding computer task, and it requires a lot of work in terms of pre-processing to transform a raw digital image into a form that is understandable by a computer as data.

The resolution of an image (or a video) is the number of pixels contained in the image (or each frame), and it is usually expressed by the number of rows and the number of colums (e.g, 256x256 means that an image has 256 rows and 256 columns of pixels).

A digital video is a sequence of frames, and each frame is a digital image. Digital videos can have different frame rates that are expressed as the number of images per second. Video standards vary, but a full-motion digital video is usually around 30 frames per second. Other frame rates can be used depending on the purpose of the video and the amount of bandwidth and storage that are available.

## Part 2: Capture Images

In this part of the lab, we will use Python to automatically capture a series of images.

##### Step 1: Setup the environment.
Import all the libraries we need to perform the task.

In [ ]:
# Install a scikit-image
#!pip install scikit-image

# Or update if you allready have it

#!pip install --upgrade scikit-image



In [1]:
#code cell 1
# import the necessary packages
import cv2
import time
from matplotlib import pyplot as plt
import numpy as np

%matplotlib inline

#from IPython.display import clear_output

###### b) Capture a sequence of images.
In order to capture image sequences, we will create an infinite loop and capture images periodically as the loop repeats. Add or move items whithin the frame. You will see the video update after a short delay. This is like a very low frame rate video. 

In order to go on to code cell 4, stop code cell 3 by clicking the button with the square icon to the left of the code cell.

In [2]:
def videoCapture (frame_rate):
    #Capture the video flux with openCV (We can change 0 to 1,2,... if multiple cameras)
    
    video_capture = cv2.VideoCapture(0)
    
    image_counter = 0
    prev_time = 0

    #Main loop, runinng unless of CTRL+C or keyboard input
    while True:
        try:
            #Read the next frame into buffer
            ret,buffer_frame = video_capture.read() # First frame
            
            #Set framerate
            time_elapsed = time.time() - prev_time
            
            if time_elapsed > 1./frame_rate:
                prev_time = time.time()

                #Display captured frame, so we can see what is happening
                cv2.imshow('img', buffer_frame)
                cv2.waitKey(1) # Image not displayed if line removed
                

        #Avoid error when stopping program
        except KeyboardInterrupt:
            print('Done')
            break

    #Stop video feed and close window
    video_capture.release()
    cv2.destroyAllWindows()


In [3]:
# Be patient, takes some time to launch in JNotebook.
#Press Jupyter Notebook STOP button to stop cell.

fps=25 # frame rate = frames per second
videoCapture(fps)

Done


## Part 3: Change Detection

#### Step 4: Detect moving objects.

In order to detect a change in the captured image, the stored background image will be subtracted from the current frame. Because of noise, the difference between the stored background and the current background will not always be exactly zero if no change has occurred. In order to avoid falsely detecting change, we will have to set a threshold that will let us decide whether the difference between the two images is small enough to be considered zero or if we think that something is actually moving in the scene.

Execute the following code. The code will loop indefinately until it is stopped.

In [4]:
#scikit-image, for comparison, SSIM algorithm
from skimage.metrics import structural_similarity as ssim

# --- VARS ---
# Threshold, if the error is under it, we trigger an alert
# 1 is perfect match between 2 images
# Found that we had to lower it to deal with noise and micro movements
similarityValue = 0.90

#Capture the video flux with openCV (We can change 0 to 1,2,... if multiple cameras)'

video_capture =  cv2.VideoCapture(0)

# First, we need to initialise the stored frame
# right now it's empty so we have nothing to compare to

# Get the actual frame
ret,buffer_frame = video_capture.read()

#Transform picture to GrayScale, easier to compare
buffer_frame = cv2.cvtColor(buffer_frame, cv2.COLOR_BGR2GRAY)
      
#Store actual frame, so now we can start comparing
previous_frame = buffer_frame

#We do not need to compare all the frames
#let's compare every 10 frames (Approx every ~0.4s for a 24fps camera)
comparison_frequency = 10

#Counter for video frames, we already got 1 image, so init at 1
image_counter = 1

#Counter for printing events with number
event_counter = 0

#Main loop, runinng unless of CTRL+C or keyboard input
while True:
    try:
        #Read the next frame into buffer
        ret,buffer_frame = video_capture.read()
    
        #If we don't have images anymore, exit
        if buffer_frame is None:
            break
        
        #Switch to gray
        buffer_frame = cv2.cvtColor(buffer_frame, cv2.COLOR_BGR2GRAY)
    
        #Compare every 10 frames
        if image_counter == comparison_frequency:
            #Get error value
            ssim_index = ssim(previous_frame, buffer_frame)
        
            #If error under threshold, trigger warning
            if ssim_index < similarityValue:
                event_counter +=1
                print('Event'+str(event_counter)+' : Something moving')   
                
              
            
            #Store current frame as new reference for comparison
            previous_frame = buffer_frame
           
            #Display captured frame, so we can see what is happening
            cv2.imshow('img', buffer_frame)
            cv2.waitKey(1) # Image not displayed if line removed
            
            #if comparison happened, reset frame counter
            image_counter=0
            
        #Increment at each loop / captured frame
        image_counter+=1
    
    #Avoid error when stopping program
    except KeyboardInterrupt:
        print('All done')
        break
        
#Stop video feed and close window
video_capture.release()
cv2.destroyAllWindows()


Event1 : Something moving
Event2 : Something moving
Event3 : Something moving
Event4 : Something moving
Event5 : Something moving
Event6 : Something moving
Event7 : Something moving
Event8 : Something moving
Event9 : Something moving
Event10 : Something moving
Event11 : Something moving
Event12 : Something moving
Event13 : Something moving
Event14 : Something moving
Event15 : Something moving
Event16 : Something moving
Event17 : Something moving
All done


## Part 4:  Application


#### Application
Digital video cameras have become inexpensive, plentiful, and practical to use. Video cameras that are connected to networks can be seen in many cities and businesses. Some of these cameras are connected to the Internet. The streams of video data that they produce are processed and analyzed at the network edge or in the Cloud. What is this video data used for?

An obvious application is for security and surveillance. Basic change detection algorithms like the one demonstrated in this lab could be used to detect the actions of people or the movement of things. Change detection can be used to detect when parking places become vacant or when store shelves are empty. In traffic management, a change detection process can be used to compare the current conditions on a highway to the expected conditions for that time and day. A differrence may indicate a traffic jam in or near the area that the camera is monitoring. This could be used to alert public safety personnel of the need to investigate road conditions.

Can you think of any other applications for analysis of video data and change detection?

> Security Camera  
> Connected doorbell  
> Animals detection / counter  
> Poacher detect  
> Autonomous cars  
> Pattern detections  
> ...

<font size='0.5'>&copy; 2017 Cisco and/or its affiliates. All rights reserved. This document is Cisco Public.<font>